In [8]:
import torch
import tempfile
import torchaudio
from transformers import pipeline
from datasets import load_dataset
import os

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3",
    chunk_length_s=20,                 
    return_timestamps=True,
    torch_dtype=torch.float16,         
    device=device
)

/home/deniska/varya/project_transcribe/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def transcribe_audio_file(audio_bytes: bytes, original_filename: str, return_timestamps: bool = False):
    """
    Транскрибирует аудиофайл, возвращая текст или чанки с таймкодами.

    :param audio_bytes: содержимое аудиофайла в байтах
    :param original_filename: имя файла (нужно для определения расширения)
    :param return_timestamps: возвращать ли таймкоды
    :return: строка текста или список чанков с таймкодами
    """
    ext = os.path.splitext(original_filename)[1].lower()

    with tempfile.NamedTemporaryFile(suffix=ext, delete=True) as temp_audio:
        temp_audio.write(audio_bytes)
        temp_audio.flush()

        # Загружаем аудио
        waveform, sample_rate = torchaudio.load(temp_audio.name)

        # Приводим к 16kHz, если нужно
        if sample_rate != 16000:
            waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)

        # Приводим к моно, если стерео
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # Whisper ожидает NumPy-массив
        audio_input = waveform.squeeze().numpy()

        result = asr_pipeline(
            audio_input,
            batch_size=8,
            return_timestamps=return_timestamps,
        )

        return result["chunks"] if return_timestamps else result["text"]


In [11]:
import yt_dlp
def download_audio_from_youtube(url: str) -> tuple[bytes, str]:
    """
    Скачивает аудио с YouTube и возвращает его как байты и имя файла.
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        output_path = os.path.join(tmpdir, "%(title)s.%(ext)s")
        ydl_opts = {
            "format": "bestaudio/best",
            "outtmpl": output_path,
            "noplaylist": True,
            "quiet": True,
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info_dict).rsplit(".", 1)[0] + ".mp3"

        with open(filename, "rb") as f:
            audio_bytes = f.read()

        return audio_bytes, os.path.basename(filename)


In [13]:
if __name__ == "__main__":
    filename = ""
    # with open(filename, "rb") as f:
    #     audio = f.read()
    youtube_url = input("Вставьте ссылку на YouTube: ").strip()

    print("\n📥 Скачиваем аудио...")
    audio, filename = download_audio_from_youtube(youtube_url)
    
    print("\n=== Только текст ===")
    text = transcribe_audio_file(audio, filename, return_timestamps=False)
    print(text)

    print("\n=== С таймкодами ===")
    chunks = transcribe_audio_file(audio, filename, return_timestamps=True)
    for chunk in chunks:
        print(chunk)


📥 Скачиваем аудио...
                                                         
=== Только текст ===
 Рома, Ютуб не вечный, рано или поздно бабосики капать перестанут, а у тебя неосвоенной профессии, не опыта работы, что будешь делать, когда золотой осёл иссякнет? И этот парень прав, чуваки, серьёзно, особенно учитывая, что я это сам и написал, чтобы у меня была подводка к видосу. А вот пруфы. Но как же мне найти нормальную работу, если я не умею ее работать? Я надеюсь, в этом мне смогут помочь два человечка. То есть это Кодер Павел с огромным рабочим хером и стажем, и Максос. Ребята будут подсказывать мне верные ответы на собеседование. Возможно, уже через неделю, ребят, наконец-таки, похлопаем, я удалю свой канал хуя. Правда, я не до конца уверен, что Максос будет подсказывать мне верные ответ Нет, нет, можно выключить. Да. Ага. Ну, я не работаю, к сожалению, камерами. Включите послучайно. Включите. включите по случайно и оставь навсегда хорошо хорошо давайте сейчас мы делаем приложе

In [ ]:
if __name__ == "__main__":
    youtube_url = input("Вставьте ссылку на YouTube: ").strip()

    print("\n📥 Скачиваем аудио...")
    audio, filename = download_audio_from_youtube(youtube_url)

    print("\n=== Только текст ===")
    text = transcribe_audio_file(audio, filename, return_timestamps=False)
    print(text)

    print("\n=== С таймкодами ===")
    chunks = transcribe_audio_file(audio, filename, return_timestamps=True)
    for chunk in chunks:
        print(chunk)

In [2]:
import os
import yt_dlp
from pathlib import Path

def download_youtube_audio(url: str):
    # Путь к папке "Загрузки" (работает на Windows/Linux/macOS)
    downloads_path = str(Path.home() / "Downloads")

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": os.path.join(downloads_path, "%(title)s.%(ext)s"),
        "noplaylist": True,
        "quiet": False,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print(f"📥 Скачиваем {url} в MP3...")
        ydl.download([url])
        print(f"✅ Скачано в: {downloads_path}")

if __name__ == "__main__":
    youtube_url = input("Вставьте ссылку на YouTube-видео: ").strip()
    download_youtube_audio(youtube_url)


📥 Скачиваем https://youtu.be/MrNbqT7-S-Y?si=CwJXMduWxFmoa1pa в MP3...
[youtube] Extracting URL: https://youtu.be/MrNbqT7-S-Y?si=CwJXMduWxFmoa1pa
[youtube] MrNbqT7-S-Y: Downloading webpage
[youtube] MrNbqT7-S-Y: Downloading tv client config
[youtube] MrNbqT7-S-Y: Downloading player afb1da51-main
[youtube] MrNbqT7-S-Y: Downloading tv player API JSON
[youtube] MrNbqT7-S-Y: Downloading ios player API JSON
[youtube] MrNbqT7-S-Y: Downloading m3u8 information
[info] MrNbqT7-S-Y: Downloading 1 format(s): 251
[download] Destination: /home/deniska/Downloads/Внушил 56-летнему танкисту, что он добывает реальный биткоин в Майнкрафте, и месяц платил ему за это.webm
[download] 100% of   64.47MiB in 00:00:09 at 6.51MiB/s   
[ExtractAudio] Destination: /home/deniska/Downloads/Внушил 56-летнему танкисту, что он добывает реальный биткоин в Майнкрафте, и месяц платил ему за это.mp3
Deleting original file /home/deniska/Downloads/Внушил 56-летнему танкисту, что он добывает реальный биткоин в Майнкрафте, и м